<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/Sonion_process_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import re

from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control Plan data REC 3000 - WK34 - 2020.xlsm'

In [3]:
import xlrd
xls = xlrd.open_workbook(path, on_demand=True)
sheet_names=xls.sheet_names()
print(xls.sheet_names())

['Master list', '39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37560', '30740', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2', '37751']


In [4]:
sheet_dict={}
xls = pd.ExcelFile(path)
for name in sheet_names:
  sheet_dict[name] = pd.read_excel(xls, name)

In [5]:
name='38682'
sheet=sheet_dict[name]

In [6]:
#sheet.to_csv('abc.xlsx')

In [7]:
# get information in each coil
#dim_name=sheet[sheet.columns[14]][22]
#max=sheet[sheet.columns[14]][23]
#min=sheet[sheet.columns[14]][24]
#date=pd.to_datetime(sheet[sheet.columns[9]][25:])
#dim_value=sheet[sheet.columns[14]][25:]

In [8]:
# find begin and end col
values_col=sheet.iloc[22,:]
values_col.reset_index(drop=True,inplace=True)
begin_col=values_col[values_col=='Kích thước\nDimension'].index
begin_col=begin_col+1
begin_col

Int64Index([14], dtype='int64')

In [10]:
end_col=values_col[values_col=='MSNV'].index
end_col

Int64Index([20], dtype='int64')

In [16]:
sheet.columns[begin_col[0]:end_col[0]]

Index(['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [105]:
df_dict={}
tolerance_dict={}
for name in sheet.columns[begin_col[0]:end_col[0]]:
  df=pd.DataFrame()
  
  #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
  df['Date']=sheet[sheet.columns[9]][25:]
  df['Date']=df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
  df['Date']=pd.to_datetime(df['Date'])
  #df['Hour']=df['Date'].dt.hour
  df['Value']=sheet[name][25:]
  df['USL']=sheet[name][23] # max
  df['LSL']=sheet[name][24] # min
  df.dropna(subset=['Value'],inplace=True)
  #UCL,LCL,nominal:
  sigma=3
  df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
  df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
  df['Mean']=df['Value'].mean()
  df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
  dim_name=sheet[name][22]
  df_dict[dim_name]=df.reset_index(drop=True)

In [106]:
df_dict['L.Tâm Dim N']

,Date,Value,USL,LSL,UCL,LCL,Mean
0,2020-08-14 17:00:00,0.012,0.025,-0.025,0.019721,-0.004197,0.007762
1,2020-08-14 17:00:00,0.006,0.025,-0.025,0.019721,-0.004197,0.007762
2,2020-08-14 17:00:00,0.009,0.025,-0.025,0.019721,-0.004197,0.007762
3,2020-08-15 08:00:00,0.011,0.025,-0.025,0.019721,-0.004197,0.007762
4,2020-08-15 08:00:00,0.008,0.025,-0.025,0.019721,-0.004197,0.007762
5,2020-08-15 08:00:00,0.008,0.025,-0.025,0.019721,-0.004197,0.007762
6,2020-08-15 16:00:00,0.005,0.025,-0.025,0.019721,-0.004197,0.007762
7,2020-08-15 16:00:00,0.004,0.025,-0.025,0.019721,-0.004197,0.007762
8,2020-08-15 16:00:00,0.002,0.025,-0.025,0.019721,-0.004197,0.007762
9,2020-08-17 09:00:00,0.016,0.025,-0.025,0.019721,-0.004197,0.007762


In [108]:
i=1
#Layout
fig = make_subplots(
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    subplot_titles=(list(df_dict.keys()))
)
for name in df_dict.keys(): #also group
  df=df_dict[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)

  df=df.groupby('Date').mean()
  #df=df.set_index('Date')
  #if start_date != '':
  #  df=df[start_date:end_date]
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df.index, y=df['Value'],mode='lines+markers',
                          name='mean ' + name,line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  fig.append_trace(go.Scatter(x=df.index, y=df['UCL'],name='UCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LCL'],name='LCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['Mean'],name='Mean '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  i=i+1
  
fig.update_layout(height=200*len(df_dict), width=1200, title_text='Line chart')
fig.show()

In [110]:
i=1
#Layout
fig = make_subplots(
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    subplot_titles=(list(df_dict.keys()))
)
for name in df_dict.keys(): #also group
  df=df_dict[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.set_index('Date')
  #Control chart 1 
  fig.append_trace(go.Box(
                          x=df.index, y=df['Value'],
                          name='mean ' + name, line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733')),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  i=i+1
  
fig.update_layout(height=200*len(df_dict), width=1200, title_text='Line chart')
fig.show()

# Combine to one

In [111]:
path='/content/drive/My Drive/Data/Sonion/Control plan and stast/Control Plan data REC 3000 - WK34 - 2020.xlsm'
import xlrd
xls = xlrd.open_workbook(path, on_demand=True)
sheet_names=xls.sheet_names()
print(xls.sheet_names())
sheet_dict={}
xls = pd.ExcelFile(path)
for name in sheet_names:
  sheet_dict[name] = pd.read_excel(xls, name)

['Master list', '39682', '38682', '30682', '37682', '30665', '30660-37660-38660', '30380-37380-38380', '37670-39670', '37662', '30650-37650 (3500T)', '37650-38655-39650', '37641', '30470-37470', '38546', '39615', '38940', '38594', '37560', '30740', '37640-38635', '30674-37674-38674 ', '30672-37672-38672', '30592', '30544-37544-38544-39544', '30538', '30530-37530-39530', '30480-37480-38481', '30370-37370-38370 ', '30750-30940-39750 _S1', '30750-30940-39750 _S2', '37750-37940_S1', '37750-37940_S2', '37751']


In [126]:
sheet_all={}
sheet_error=[]
for name_sheet in sheet_names[1:]: # all sheet
  #print(name_sheet)
  sheet=sheet_dict[name_sheet]

  # find begin and end col
  values_col=sheet.iloc[22,:]
  values_col.reset_index(drop=True,inplace=True)
  begin_col=values_col[values_col=='Kích thước\nDimension'].index
  begin_col=begin_col+1
  end_col=values_col[values_col=='MSNV'].index
  #print(begin_col,end_col)
 
  df_dict={} # add all value, USL, LSL, UCL... in sheet
  try:
    for name in sheet.columns[begin_col[0]:end_col[0]]:
      
        df=pd.DataFrame()
        
        #tolerance_dict[sheet[name][22]]=[sheet[name][24],sheet[name][23]]
        df['Date']=sheet[sheet.columns[9]][25:]
        df['Date']=df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
        df['Date']=pd.to_datetime(df['Date'])
        #df['Hour']=df['Date'].dt.hour
        df['Value']=sheet[name][25:]
        df['USL']=sheet[name][23] # max
        df['LSL']=sheet[name][24] # min
        df.dropna(subset=['Value'],inplace=True)
        #UCL,LCL,nominal:
        sigma=3
        df['UCL']=df['Value'].mean() + df['Value'].std()*sigma
        df['LCL']=df['Value'].mean() - df['Value'].std()*sigma
        df['Mean']=df['Value'].mean()
        df[df.columns[1:]]=df[df.columns[1:]].astype('float32')
        dim_name=sheet[name][22]
        df_dict[dim_name]=df.reset_index(drop=True)
  except:
    sheet_error.append(name_sheet)
    continue
  sheet_all[name_sheet]=df_dict
print(sheet_error)

['37560', '30740', '37751']


In [ ]:
sheet_all['30380-37380-38380']

In [130]:
for sheet_name in sheet_all.keys():
  df_dict=sheet_all[sheet_name]

  i=1
  #Layout
  fig = make_subplots(
      rows=len(df_dict), cols=1,
      #shared_xaxes=True, # share same axis
      #vertical_spacing=0.05, # adjust spacing between charts
      #column_widths=[0.8, 0.2],
      subplot_titles=(list(df_dict.keys()))
  )
  for name in df_dict.keys(): #also group
    df=df_dict[name].copy()
    df=df.sort_values(by=['Date'])
    for a in df.columns[1:]:
      df[a] = df[a].round(decimals=3)

    df=df.groupby('Date').mean()
    #df=df.set_index('Date')
    #if start_date != '':
    #  df=df[start_date:end_date]
    #Control chart 1 
    fig.append_trace(go.Scatter(
                            x=df.index, y=df['Value'],mode='lines+markers',
                            name='mean ' + name,line=dict( color='#4280F5')
                            ),row=i, col=1
                  )
    #USL, LSL
    fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
    #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
    # UCL, LCL
    fig.append_trace(go.Scatter(x=df.index, y=df['UCL'],name='UCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['LCL'],name='LCL '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    fig.append_trace(go.Scatter(x=df.index, y=df['Mean'],name='Mean '+name, line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
    i=i+1
    
  fig.update_layout(height=200*len(df_dict), width=1200, title_text=sheet_name)
  fig.show()